In [1]:
from datetime import datetime

import pandas as pd
import numpy as np

In [20]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)

In [3]:
df = pd.read_csv('btc_minutes_data_labeled')
df.shape

(2602679, 8)

In [12]:
df_len = 2602679

In [13]:
ten_percent_df = 260300

In [51]:
counter = 1
columns = [
    'time',
    'open',
    'close',
    'high',
    'low',
    'volume',
    'label'
]
for i in range(0, df_len, ten_percent_df):
    if counter == 1:
        df = pd.read_csv('btc_minutes_data_labeled', skiprows=i, nrows=ten_percent_df, index_col=0)
        
        df = calculate_indicators(df, intervals)

        df.to_csv(f'batch_{str(counter)}_indicators')
        
        del df
        
    else:
        df = pd.read_csv('btc_minutes_data_labeled', skiprows=i-21, nrows=ten_percent_df+21, index_col=0, header=None)
        df.columns = columns
        
        df = calculate_indicators(df, intervals)
        
        df.iloc[22:,:].to_csv(f'batch_{str(counter)}_indicators')
        
        del df
        
    counter += 1

starting sma
Duration: 0:00:00.070631
starting ema
Duration: 0:00:00.199261
starting wma
Duration: 0:00:11.675239
starting hull
Duration: 0:00:24.789845
starting tema
Duration: 0:00:00.699637
starting rsi
Duration: 0:00:03.426962
starting williams_r
Duration: 0:00:00.939917
starting CCI
Duration: 0:00:00.353089
starting CMO
Duration: 0:00:03.294820
starting MACD
Duration: 0:00:00.808833
starting PPO
Duration: 0:00:00.825117
starting ROC
Duration: 0:00:01.465645
starting stochastic
Duration: 0:00:01.128783
starting sma
Duration: 0:00:00.077455
starting ema
Duration: 0:00:00.210340
starting wma
Duration: 0:00:11.906176
starting hull
Duration: 0:00:23.895432
starting tema
Duration: 0:00:00.640447
starting rsi
Duration: 0:00:03.317371
starting williams_r
Duration: 0:00:00.948564
starting CCI
Duration: 0:00:00.321138
starting CMO
Duration: 0:00:03.344157
starting MACD
Duration: 0:00:00.830096
starting PPO
Duration: 0:00:00.866126
starting ROC
Duration: 0:00:01.490552
starting stochastic
Dur

In [7]:
df.head()

,open,close,high,low,volume,label
0,746.99,747.00,747.00,746.99,2.000000,NaN
1,743.89,744.89,744.89,743.89,0.020695,NaN
2,745.01,745.00,745.01,745.00,0.900000,NaN
3,747.00,747.30,747.30,747.00,2.500000,NaN
4,744.87,744.87,744.87,744.87,1.230769,NaN


# Technical Indicators

1. Relative Strength Index (RSI) --> Oscilates between 0 and 100, if over 70 = overbought region, if under 30 = oversold region
2. Williams %R --> Momentum indicator, oscilates between -100 and 0, under -80 = oversold, over -20 = overbought
3. Simple Moving Average (SMA) --> moving average of price for a given period, intersections of SMAs of different periods (MA crosses) used to identify trends
4. Exponential Moving Average (EMA) --> moving average for a period where most recent data has more weight
5. Weighted Moving Average (WMA) --> similar to above EMA except that for WMA importance of closing price decreases lineraly when moving back where EMA decreases exponentially
6. Hull Moving Average (HMA) --> type of moving average to reduce lag associated with SMA, tends to perform better than EMA and WMA
7. Triple Exponential Moving Average (TEMA) --> type of EMA indicator that reduces minor price fluctuations and filters out volatitity
8. Commodity Channel Index (CCI) --> compares current price and average price over a period of time, oscillates about 75% between -100 and 100 and 25% it passes its range values
9. Chande Momentum Oscilator Indicator (CMO) --> momentum similar to RSI and Stochastic Oscillator, oscillates between -100 and 100, if over 50 = over bought and under -50 = oversold
10. Moving Average Convergence and Divergence (MACD) --> shows trends and made up of two parts, MACD and signal line, if MACD crosses signal upward then buy and if downward sell
11. Percentage Price Oscilator (PPO) --> very similar to MACD
12. Rate of Change (ROC) --> illustrates the speed of price change over a period of time
13. Chaikin Money Flow Indicator (CMFI) --> measure money flow volume over a period of time, fluctuates between 1 and -1, closer to 1 = higher buying pressure, -1 = selling pressure higher
14. Directional Movement Indicator (DMI) --> shows trend stregth and direction, made up of 3 seperate indicators, Average Directional Index (ADX), Plus Directional Indicator (+DI), Minus Directional Indicator (-DI)
15. Parabolic SAR (SAR) --> try to determine points of potential stops and reverses, 3 emelements, Previous SAR (PSAR), Extreme Point (EP), and Acceleration Factor (AF)

In [37]:
def get_sma(df, intervals):
    """
    simple moving average of price for a given period or interval, used for identifing trends.
    
    df: Pandas DataFrame
    intervals: list of ints, list of time periods
    
    return: DataFrame with new columns for each period in list
    """
    print('starting sma')
    start_time = datetime.now()
    for period in intervals:
        df["SMA_" + str(period)] = df['close'].rolling(window=period).mean()
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    return df


def get_ema(df, intervals):
    """
    Exponential moving average of price for a given period or interval, used for identifing trends.
    
    df: Pandas DataFrame
    intervals: list of ints, list of time periods
    
    return: DataFrame with new columns for each period in list
    """
    print('starting ema')
    start_time = datetime.now()
    
    for period in intervals:
        df["EMA_" + str(period)] = df['close'].ewm(span=period).mean()
    
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    
    return df


def get_wma(df, intervals):
    """
    weighted moving average of price for a given period or interval, used for identifing trends.
    
    df: Pandas DataFrame
    col_name: string, Column in DataFrame that we want to calculate the average on
    intervals: list of ints, list of time periods
    
    return: DataFrame with new columns for each period in list
    """
    print('starting wma')
    start_time = datetime.now()
    
    for period in intervals:
        weights = np.arange(1, period + 1)        
        df["WMA_" + str(period)] = df['close'].rolling(window=period).apply(lambda x: np.dot(x, weights) / weights.sum(), raw=True)
    
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    
    return df


def get_hma(df, intervals, wma_cols=True):
    """
    Hull Moving Average, type of moving average that reduces lag.
    First, calculate the Weighted Moving Average for the period on 'close' price.
    Second, calculate the weighted Moving Average for n/2 periods on 'close' price and multiple the result by 2.
    Third, calculate the Weighted Moving Average for sqrt(n) periods using the values from step 2.
    
    df: Pandas DataFrame, should have Open, High, Lose, Close price data but only Close is needed
    intervals: list of ints, list of time periods to calculate
    wma_cols: Default is False, check to see if Weighted Moving Averages are already calculated, if so use those values for step 1, if not calculate them.
    
    return: Pandas DataFrame df with new Hull Moving Average columns added for each period in intervals list.
    """
    print('starting hull')
    start_time = datetime.now()
    for period in intervals:
        #  wma for period using wma_columns already in DataFrame
        if wma_cols is True:
            wma = df["WMA_" + str(period)]
        # if wma's have not been calculated yet calculate them 
        else:
            # set weights for period
            weights1 = np.arange(1, period + 1)        
            wma1 = df['close'].rolling(window=period).apply(lambda x: np.dot(x, weights1) / weights1.sum(), raw=True)


        # calculate wma for period/2
        # set weights for period/2
        weights2 = np.arange(1, int(period/2)+1)
        wma2 = df['close'].rolling(window=int(period/2)).apply(lambda x: np.dot(x, weights2) / weights2.sum(), raw=True)
        # multiple wma2 by 2
        wma2 = wma2.apply(lambda x: 2*x)

        # subtract wma2 - wma
        wma3 = wma2 - wma

        # calculate wma for sqrt(period) using wma
        # calculate weights for sqrt(period)
        weights3 = np.arange(1, int(np.sqrt(period))+1)
        # calculate Hull Moving Average 
        df["HMA_" + str(period)] = wma3.rolling(window=int(np.sqrt(period))).apply(lambda x: np.dot(x, weights3) / weights3.sum(), raw=True)
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))    
    return df


def get_tema(df, intervals):
    """
    Triple Exponential Moving Average (TEMA), type of moving average that tries to filter out volatility and reduce impact of minor price fluctuations.
    
    df: Pandas DataFrame
    col_name: string, Column in DataFrame that we want to calculate the average on
    intervals: list of ints, list of time periods to calculate
    
    return: Pandas DataFrame df with new Triple Exponential Moving Average columns added for each period in intervals list. 
    """
    print('starting tema')
    start_time = datetime.now()
    # formula = (3 * EMA1) - (3 * EMA2) + EMA3
    # EMA1 = Exponential Moving Average (EMA)
    # EMA2 = EMA of EMA1
    # EMA3 = EMA of EMA2

    for period in intervals:
        ema1 = df['close'].ewm(span=period, adjust=False).mean()
        ema2 = ema1.ewm(span=period, adjust=False).mean()
        ema3 = ema2.ewm(span=period, adjust=False).mean()

        df['TEMA_' + str(period)] = ( 3 * ema1 ) - ( 3 * ema2 ) + ema3
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    return df


def get_rsi(df, intervals, sma=True):
    """
    Oscillator type indicator that shows historical strength and weakness of price, will oscillate between 0 and 100
    Typically if greater than 70 considered overbought, if under 30 considered oversold
    
    df: Pandas DataFrame 
    col_name: string, Column name to calculate RSI on
    intervals: list of ints, list of time periods
    
    return: Copy of original DataFrame with new columns for each period in the list
    """
    print('starting rsi')
    start_time = datetime.now()
    # create series of price differences of the chosen column
    delta = df['close'].diff()

    # split up and down
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0

    # default is simple moving average to get average gains and losses
    if sma is True:
        for period in intervals:
            roll_up = up.rolling(window=period).mean()
            roll_down = down.abs().rolling(window=period).mean()
            RS = roll_up / roll_down
            df["RSI_" + str(period)] = 100.0 - (100 / (1.0 + RS))
            
    # if sma is False use exponential moving average to calculate average gains and losses
    else:
        for period in intervals:
            roll_up = up.ewm(span=period).mean()
            roll_down = down.abs().ewm(span=period).mean()
            RS = roll_up / roll_down
            df["RSI_" + str(period)] = 100.0 - (100 / (1.0 + RS))
    
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    # Return copy of original dataframe with new columns for RSI
    return df


def get_will_r(df, intervals):
    """
    Momentum oscillator indicator to determine overbought and underbough conditions
    Ranges from -100 to 0, below -80 = oversold, above -20 = overbought
    
    df: Pandas DataFrame, must have high, low and close price data
    intervals: list of ints, list of time periods
    
    return: Copy of original DataFrame with new columns for each period in the list
    """
    print('starting williams_r')
    start_time = datetime.now()
    
    # get indicator for each time period in list of intervals
    for period in intervals:
        # get highest high during period
        highest = df['high'].rolling(window=period).max()

        # get lowest low for period
        lowest = df['low'].rolling(window=period).min()

        # get last close
        last_close = df['close'].rolling(window=period).apply(lambda x: x[-1], raw=True)

        # williams_r formula
        # (highest high / last close) / (Highest High / lowest low) * -100
        df["williamsR_" + str(period)] = ((highest - last_close) / (highest - lowest)) * -100
        
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    return df


def get_cci(df, intervals, constant=0.015):
    """
    Commodity Channel Index (CCI), compares current prices and the average price over a period of time.
    About 75% of the time it oscillates between -100 and 100, 25% of the time it passes out of this range.
    
    Equation:
    typical price = (high + low + close ) / 3
    ( typical price - 20 period sma of typical price ) / ( constant * mean deviation )
    
    df: Pandas DataFrame, need high, low and close prices
    intervals: list of ints, list of time periods
    constant: int, defalut = 0.015, constant is multiplied by mean deviation
    
    return: df, Pandas DataFrame with new CCI columns for the time periods in the interval list
    """
    print('starting CCI')
    start_time = datetime.now()
    for period in intervals:
        # calculate typical price
        tp = (df['high'] + df['low'] + df['close']) / 3
        
        # use typical price to calculate CCI
        df['CCI_' + str(period)] = pd.Series((tp - tp.rolling(window=period).mean()) / (constant * tp.rolling(window=period).std()))
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))  
    return df

def get_cmo(df, intervals):
    """
    Chande Momentum Oscillator Indicator (CMO)
    Oscillates between -100 and 100, over 50 = over bought, under -50 = over sold
    Equation: ( (sum of up - sum of down) / (sum of up + sum of down) ) * 100
    
    df: Pandas DataFrame
    intervals, list of ints, list of periods
    
    return: df, Pandas DataFrame with new CMO columns for each period in intervals
    """
    print('starting CMO')
    start_time = datetime.now()
    delta = df['close'].diff()

    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0

    for period in intervals:
        up_sum = up.rolling(window=period).sum()
        down_sum = down.abs().rolling(window=period).sum()

        df['CMO_' + str(period)] = ( (up_sum - down_sum) / (up_sum + down_sum)) * 100
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    return df

def get_macd(df, intervals):
    """
    Moving Average Convergence and Divergence (MACD)
    Two equations:
    MACD line = ( 12 period ema - 26 period ema )
    Signal Line = 9 period ema of MACD Line

    try: get ema for intervals, then on each interval calculate the MACD Line and Signal Line
    then calculate the difference between MACD Line and Signal Line
    if MACD Line crosses Signal Line in upward direction = buy, if cross from downward direction = sell
    
    df: Pandas DataFrame, needs to have closing price data
    intervals: list of ints, list of periods
    
    return: df, Pandas DataFrame with MACD_diff columns for each period in the list of intervals
    """
    print('starting MACD')
    start_time = datetime.now()
    for period in intervals:
        ema_n = df['close'].ewm(span=period, adjust=False).mean()
        ema_12 = ema_n.ewm(span=12, adjust=False).mean()
        ema_26 = ema_n.ewm(span=26, adjust=False).mean()

        macd = (ema_12 - ema_26)

        signal = macd.ewm(span=9, adjust=False).mean()

        df['MACD_dif_' + str(period)] = macd - signal
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    return df


def get_ppo(df, intervals):
    """
    Percentage Price Oscillator (PPO), similar to MACD
    Two equations:
    PPO = ( (12 period ema - 26 period ema) / 26 period ema ) * 100
    Signal Line = 9 period ema of PPO

    try: get ema for intervals, then on each interval calculate the PPO and Signal Line
    then calculate the difference between PPO and Signal Line
    if PPO crosses Signal Line in upward direction = buy, if cross from downward direction = sell
    
    df: Pandas DataFrame, needs to have closing price data
    intervals: list of ints, list of periods
    
    return: df, Pandas DataFrame with PPO_diff columns for each period in the list of intervals
    """
    print('starting PPO')
    start_time = datetime.now()
    for period in intervals:
        ema_n = df['close'].ewm(span=period, adjust=False).mean()
        ema_12 = ema_n.ewm(span=12, adjust=False).mean()
        ema_26 = ema_n.ewm(span=26, adjust=False).mean()

        ppo = ((ema_12 - ema_26) / ema_26) * 100

        signal = ppo.ewm(span=9, adjust=False).mean()

        df['PPO_dif_' + str(period)] = ppo - signal
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    return df


def get_roc(df, intervals):
    """
    Rate of Change (ROC), indicates the speed of price change over a period of time.
    equation: ( (Latest Close - Previous Close) / Previous Close ) * 100
    
    df: Pandas DataFrame, needs closing price
    intervals: list of ints, list of periods to calculate ROC
    
    return: df, Pandas DataFrame with new columns for each period in list of intervals
    """
    print('starting ROC')
    start_time = datetime.now()
    for period in intervals:
        previous = df['close'].rolling(window=period).apply(lambda x: x[0], raw=True)
        latest = df['close'].rolling(window=period).apply(lambda x: x[-1], raw=True)
        df["ROC_" + str(period)] = ( (latest - previous) / previous ) * 100
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time)) 
    return df


def get_cmf(df, intervals):
    """
    Chaikin Money Flow Indicator (CMFI) fluctuates between -1 and 1
    measure money flow volume over a period of time
    equations: 3 parts
    Multiplier = ( (close - low) - (high - close) ) / (high - low)
    Money Flow Volume (MFV) = Volume * Multiplier
    n period CMF = (n period sum of MFV) / ( n period sum of Volume)
    
    df: Pandas DataFrame, needs high, low, close and volume data
    intervals: list of ints, list of periods to calculate CMF on
    
    return: df, Pandas DataFrame with new columns for each period in list of intervals
    """
    print('starting CMF')
    start_time = datetime.now()
    for period in intervals:
        multiplier = (df['close'] - df['low']) - (df['high'] - df['close']) / (df['high'] - df['low'])
        money_flow_volume = df['volume'] * multiplier
        df['CMF_' + str(period)] = money_flow_volume.rolling(window=period).sum() / df['volume'].rolling(window=period).sum()
        df['CMF_' + str(period)].fillna(0, inplace=True)
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    return df


def get_atr(df, intervals):
    """
    Average True Range measures market volatility, first solve true range then take an average of true range over period n
    
    True range: take the values below and find the maximum
    1. current high - previous low
    2. absolute value of current high - previous close
    3. absolute value of current low - previous close
    
    Once we have true range find the average for period n
    
    df: Pandas DataFrame with open, high, low, close, volume data
    intervals: list of integers, list of periods to calculate averages on
    
    return: Pandas DataFrame with new average true range column for each period in intervals list
    """
    print('starting atr')
    start_time = datetime.now()
    
    for i in range(df.index[0], len(df)):
        if i == df.index[0]:
            pass
        else:
            candle_range = df.at[i, 'high'] - df.at[i,'low']
            high_close_range = abs(df.at[i,'high'] - df.at[i-1,'close'])
            low_close_range = abs(df.at[i,'low'] - df.at[i-1,'close'])
            
            df.at[i,'true_range'] = max(candle_range, high_close_range, low_close_range)
    
    for period in intervals:
        df['avg_tr_'+str(period)] = df['true_range'].rolling(window= period).mean()
        
    df.drop(columns='true_range', inplace=True)
    
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    
    return df


def get_stochastic(df, intervals):
    """
    Stochastic Oscillator
    formula ( (current close - lowest low in period) / (highest high in period - lowest low in period) ) * 100
    ranges between 0 - 100 similar to RSI greater than 80 = overbought, less than 20 = oversold
    note: descided to take the exponential moving average of the indicator
    
    df: Pandas DataFrame, needs, open, close, low, high, and volume
    intervals: list of integers, list of time periods to calculate indicator over
    
    return: Pandas DataFrame with new columns for Stochastic Oscillator over the time periods in the list of intervals
    """

    print('starting stochastic')
    start_time = datetime.now()
    
    for period in intervals:
        # get highest high during period
        highest = df['high'].rolling(window=period).max()

        # get lowest low for period
        lowest = df['low'].rolling(window=period).min()

        # get last close
        last_close = df['close'].rolling(window=period).apply(lambda x: x[-1], raw=True)

        # calculate indicator
        oscillator = ((last_close - lowest) / (highest - lowest)) * 100

        # take exponential moving average for each period
        df['stoch_os_'+str(period)] = oscillator.ewm(span=period, adjust=False).mean()
    
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    
    return df

In [50]:
# set list of time periods 6 through 18 for 13 periods, will change to 15 when CMF and ATR are fixed
intervals = [ i + 6 for i in range(13) ]
print(intervals)

[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


In [40]:
def calculate_moving_avg_indicators(df, intervals):
    get_sma(df, intervals)
    get_ema(df, intervals)
    get_wma(df, intervals)
    get_hma(df, intervals)
    get_tema(df, intervals)
        
    return df

In [41]:
def calculat_momentum_indicators(df, intervals):
    get_rsi(df, intervals)
    get_will_r(df, intervals)
    get_cci(df, intervals)
    get_cmo(df, intervals)
    get_macd(df, intervals)
        
    return df

In [42]:
def calculate_last_indicators(df, intervals):
    get_atr(df, intervals)
    get_ppo(df, intervals)
    get_roc(df, intervals)
    # get_cmf(df, intervals) --> CMF broken, need to fix changing to 14 x 14 for now
    get_stochastic(df, intervals)
    
    return df

In [48]:
def calculate_indicators(df, intervals):
    get_sma(df, intervals)
    get_ema(df, intervals)
    get_wma(df, intervals)
    get_hma(df, intervals)
    get_tema(df, intervals)
    get_rsi(df, intervals)
    get_will_r(df, intervals)
    get_cci(df, intervals)
    get_cmo(df, intervals)
    get_macd(df, intervals)
    # get_atr(df, intervals)
    get_ppo(df, intervals)
    get_roc(df, intervals)
    get_stochastic(df, intervals)
    # get_cmf(df, intervals) --> CMF broken, need to fix changing to 14 x 14 for now
    
    return df

In [14]:
df = calculat_momentum_indicators(df, intervals)

starting rsi
Duration: 0:00:37.343570
starting williams_r
Duration: 0:00:11.658034
starting CCI
Duration: 0:00:03.367812
starting CMO
Duration: 0:00:35.503695
starting MACD
Duration: 0:00:11.506034


In [15]:
df = calculate_last_indicators(df, intervals)

starting PPO
Duration: 0:00:12.700052
starting ROC
Duration: 0:00:16.848195
starting stochastic
Duration: 0:00:14.478039


In [16]:
df.shape

(2602679, 175)

In [17]:
df.tail()

,open,close,high,low,volume,label,SMA_6,SMA_7,SMA_8,SMA_9,SMA_10,SMA_11,SMA_12,SMA_13,SMA_14,SMA_15,SMA_16,SMA_17,SMA_18,EMA_6,EMA_7,EMA_8,EMA_9,EMA_10,EMA_11,EMA_12,EMA_13,EMA_14,EMA_15,EMA_16,EMA_17,EMA_18,WMA_6,WMA_7,WMA_8,WMA_9,WMA_10,WMA_11,WMA_12,WMA_13,WMA_14,WMA_15,WMA_16,WMA_17,WMA_18,HMA_6,HMA_7,HMA_8,HMA_9,HMA_10,HMA_11,HMA_12,HMA_13,HMA_14,HMA_15,HMA_16,HMA_17,HMA_18,TEMA_6,TEMA_7,TEMA_8,TEMA_9,TEMA_10,TEMA_11,TEMA_12,TEMA_13,TEMA_14,TEMA_15,TEMA_16,TEMA_17,TEMA_18,RSI_6,RSI_7,RSI_8,RSI_9,RSI_10,RSI_11,RSI_12,RSI_13,RSI_14,RSI_15,RSI_16,RSI_17,RSI_18,williamsR_6,williamsR_7,williamsR_8,williamsR_9,williamsR_10,williamsR_11,williamsR_12,williamsR_13,williamsR_14,williamsR_15,williamsR_16,williamsR_17,williamsR_18,CCI_6,CCI_7,CCI_8,CCI_9,CCI_10,CCI_11,CCI_12,CCI_13,CCI_14,CCI_15,CCI_16,CCI_17,CCI_18,CMO_6,CMO_7,CMO_8,CMO_9,CMO_10,CMO_11,CMO_12,CMO_13,CMO_14,CMO_15,CMO_16,CMO_17,CMO_18,MACD_dif_6,MACD_dif_7,MACD_dif_8,MACD_dif_9,MACD_dif_10,MACD_dif_11,MACD_dif_12,MACD_dif_13,MACD_dif_14,MACD_dif_15,MACD_dif_16,MACD_dif_17,MACD_dif_18,PPO_dif_6,PPO_dif_7,PPO_dif_8,PPO_dif_9,PPO_dif_10,PPO_dif_11,PPO_dif_12,PPO_dif_13,PPO_dif_14,PPO_dif_15,PPO_dif_16,PPO_dif_17,PPO_dif_18,ROC_6,ROC_7,ROC_8,ROC_9,ROC_10,ROC_11,ROC_12,ROC_13,ROC_14,ROC_15,ROC_16,ROC_17,ROC_18,stoch_os_6,stoch_os_7,stoch_os_8,stoch_os_9,stoch_os_10,stoch_os_11,stoch_os_12,stoch_os_13,stoch_os_14,stoch_os_15,stoch_os_16,stoch_os_17,stoch_os_18
2602674,6700.400000,6700.0,6700.400000,6700.0,0.440617,NaN,6702.896683,6703.040014,6703.235012,6703.143557,6702.969202,6702.699274,6702.116001,6702.107078,6702.085144,6702.066134,6702.062001,6702.052471,6702.044001,6701.956807,6702.114802,6702.248931,6702.376094,6702.505326,6702.640589,6702.782916,6702.931858,6703.086367,6703.245290,6703.407610,6703.572536,6703.739515,6702.011443,6702.268586,6702.483347,6702.615389,6702.679719,6702.682978,6702.595751,6702.525940,6702.467168,6702.417038,6702.375269,6702.339403,6702.308308,6699.648567,6699.421107,6700.349383,6701.036244,6701.879666,6701.960532,6702.579969,6702.669733,6703.102582,6703.166098,6703.774872,6703.810433,6703.806433,6700.436411,6700.846884,6701.194149,6701.437990,6701.572137,6701.611840,6701.582253,6701.510054,6701.418532,6701.325432,6701.242603,6701.176639,6701.129938,32.432432,30.508475,41.378678,45.333333,50.000000,60.386473,46.296296,46.691176,46.691176,46.350365,46.545455,46.545455,38.905775,-100.000000,-100.000000,-100.000000,-100.000000,-71.428571,-61.983471,-57.251908,-57.251908,-57.251908,-57.251908,-57.251908,-57.251908,-57.251908,-103.060878,-115.587841,-126.836430,-124.184981,-104.114492,-77.925163,-54.321981,-58.641157,-59.916452,-61.174963,-65.136724,-66.278054,-67.547178,-35.135135,-38.983051,-17.242644,-9.333333,0.000000,20.772947,-7.407407,-6.617647,-6.617647,-7.299270,-6.909091,-6.909091,-22.188450,0.283513,0.242929,0.201517,0.162991,0.129322,0.101282,0.078882,0.061695,0.049076,0.040304,0.034666,0.031508,0.030251,0.004220,0.003615,0.002997,0.002423,0.001920,0.001502,0.001168,0.000912,0.000724,0.000593,0.000509,0.000462,0.000443,-0.049229,-0.058175,-0.068610,-0.035986,-0.020891,0.000000,0.064220,-0.029842,-0.026858,-0.026858,-0.029842,-0.028350,-0.028350,29.610191,34.467967,40.247545,46.688231,53.539710,54.534204,53.466458,51.198951,48.552461,45.458447,42.290835,38.763921,35.237560
2602675,6700.000000,6681.5,6700.008115,6675.1,20.049320,NaN,6699.263350,6699.840014,6700.347512,6700.820011,6700.979202,6701.017456,6700.932668,6700.530155,6700.635144,6700.712801,6700.780751,6700.852471,6700.910668,6696.112005,6696.961101,6697.638057,6698.200875,6698.686176,6699.117157,6699.508621,6699.870164,6700.208185,6700.527129,6700.830244,6701.120032,6701.398514,6695.898105,6696.883582,6697.653344,6698.286678,6698.776228,6699.149766,6699.424058,6699.582072,6699.722482,6699.846272,6699.956210,6700.055795,6700.145781,6690.219671,6689.476972,6692.068661,6693.946343,6695.777629,6695.615085,6697.178892,6697.139855,6698.316246,6698.282787,6700.212515,6700.196968,6

In [18]:
df.head(25)

,open,close,high,low,volume,label,SMA_6,SMA_7,SMA_8,SMA_9,SMA_10,SMA_11,SMA_12,SMA_13,SMA_14,SMA_15,SMA_16,SMA_17,SMA_18,EMA_6,EMA_7,EMA_8,EMA_9,EMA_10,EMA_11,EMA_12,EMA_13,EMA_14,EMA_15,EMA_16,EMA_17,EMA_18,WMA_6,WMA_7,WMA_8,WMA_9,WMA_10,WMA_11,WMA_12,WMA_13,WMA_14,WMA_15,WMA_16,WMA_17,WMA_18,HMA_6,HMA_7,HMA_8,HMA_9,HMA_10,HMA_11,HMA_12,HMA_13,HMA_14,HMA_15,HMA_16,HMA_17,HMA_18,TEMA_6,TEMA_7,TEMA_8,TEMA_9,TEMA_10,TEMA_11,TEMA_12,TEMA_13,TEMA_14,TEMA_15,TEMA_16,TEMA_17,TEMA_18,RSI_6,RSI_7,RSI_8,RSI_9,RSI_10,RSI_11,RSI_12,RSI_13,RSI_14,RSI_15,RSI_16,RSI_17,RSI_18,williamsR_6,williamsR_7,williamsR_8,williamsR_9,williamsR_10,williamsR_11,williamsR_12,williamsR_13,williamsR_14,williamsR_15,williamsR_16,williamsR_17,williamsR_18,CCI_6,CCI_7,CCI_8,CCI_9,CCI_10,CCI_11,CCI_12,CCI_13,CCI_14,CCI_15,CCI_16,CCI_17,CCI_18,CMO_6,CMO_7,CMO_8,CMO_9,CMO_10,CMO_11,CMO_12,CMO_13,CMO_14,CMO_15,CMO_16,CMO_17,CMO_18,MACD_dif_6,MACD_dif_7,MACD_dif_8,MACD_dif_9,MACD_dif_10,MACD_dif_11,MACD_dif_12,MACD_dif_13,MACD_dif_14,MACD_dif_15,MACD_dif_16,MACD_dif_17,MACD_dif_18,PPO_dif_6,PPO_dif_7,PPO_dif_8,PPO_dif_9,PPO_dif_10,PPO_dif_11,PPO_dif_12,PPO_dif_13,PPO_dif_14,PPO_dif_15,PPO_dif_16,PPO_dif_17,PPO_dif_18,ROC_6,ROC_7,ROC_8,ROC_9,ROC_10,ROC_11,ROC_12,ROC_13,ROC_14,ROC_15,ROC_16,ROC_17,ROC_18,stoch_os_6,stoch_os_7,stoch_os_8,stoch_os_9,stoch_os_10,stoch_os_11,stoch_os_12,stoch_os_13,stoch_os_14,stoch_os_15,stoch_os_16,stoch_os_17,stoch_os_18
0,746.99,747.0000,747.0000,746.9900,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,743.89,744.8900,744.8900,743.8900,0.020695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,745.769167,745.794286,745.813125,745.827778,745.839500,745.849091,745.857083,745.863846,745.869643,745.874667,745.879063,745.882941,745.886389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,745.658950,745.780156,745.882771,745.970320,746.045665,746.111065,746.168293,746.218746,746.263532,746.303535,746.339471,746.371920,746.401362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.038473,-0.033664,-0.029923,-0.026931,-0.024483,-0.022443,-0.020716,-0.019236,-0.017954,-0.016832,-0.015842,-0.014962,-0.014174,-0.005151,-0.004507,-0.004006,-0.003605,-0.003278,-0.003004,-0.002773,-0.002575,-0.002404,-0.002253,-0.002121,-0.002003,-0.001898,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,745.01,745.0000,745.0100,745.0000,0.900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,745.423394,745.450811,745.471865,745.488525,745.502027,745.513187,745.522564,745.530551,745.537436,745.543432,745.548700,745.553364,745.557523,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,745.069763,745.176133,745.279168,745.375808,745.465054,745.546875,745.621671,745.690012,745.752513,745.809768,745.

In [ ]:
df.to_csv("btc_data_labels_indicators")